# Heart Disease Classification - XGBoost (XGB)

## 1. Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. Data Preprocessing

In [2]:
# Load dataframe (df)
df = pd.read_csv('heart.csv')
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values

print(X.shape, y.shape)

(303, 13) (303,)


In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
# Check for missing values
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [6]:
# Describe dataframe
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [7]:
# Split dataframe (df) into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)

In [8]:
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 3. Define XGB model

In [9]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [10]:
# Accuracy scoring
xgb_model = XGBClassifier(max_depth=1, learning_rate=0.17, gamma=0, 
                          colsample_bytree=0.31, n_estimators=28, 
                          min_child_weight=11, subsample=1.0, random_state=0)

# ROC_AUC scoring
# xgb_model = XGBClassifier(max_depth=1, learning_rate=0.1, gamma=0, 
#                           colsample_bytree=0.16, n_estimators=25, 
#                           min_child_weight=10, subsample=1.0, random_state=0)

xgb_model.fit(X_train, y_train)


XGBClassifier(colsample_bytree=0.31, learning_rate=0.17, max_depth=1,
              min_child_weight=11, n_estimators=28, subsample=1.0)

## 4. Evaluate Model

In [11]:
# Predict test set results
xgb_pred = xgb_model.predict(X_test)

# Create the confusion matrix
# TN = cm[0][0]  # True Negative
# FP = cm[0][1]  # False Positive
# FN = cm[1][0]  # False Negative
# TP = cm[1][1]  # True Positive
print('Confusion Matrix: ')
cm = confusion_matrix(y_test, xgb_pred)
print(cm)
print('')
print("Val. Accuracy: %.3f%% " % (accuracy_score(y_test, xgb_pred) * 100))
print("Val. ROC AUC: %.3f%% " % (roc_auc_score(y_test, xgb_pred) * 100))
print('')

# accuracy = (TP + TN)/(TN + FP + FN + TP)
# precision = TP/(TP + FP)
# recall = TP/(TP + FN)
# f1_score = (2 * precision * recall)/(precision + recall)
print('Classification Report: ')
print(classification_report(y_test, xgb_pred))

Confusion Matrix: 
[[23  4]
 [ 4 30]]

Val. Accuracy: 86.885% 
Val. ROC AUC: 86.710% 

Classification Report: 
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        27
           1       0.88      0.88      0.88        34

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61



In [12]:
# Apply k-Fold Cross Validation on the training set
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 0)
scores = cross_val_score(xgb_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# scores = cross_val_score(xgb_model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

# Force scores to be positive
scores = abs(scores)
print("Mean Accuracy: %.3f%%" % (np.mean(scores) * 100))
print("Standard Deviation: %.3f%%" % np.std(scores))
print("Min Accuracy: %.3f%%" % (np.min(scores) * 100))
print("Max Accuracy: %.3f%%" % (np.max(scores) * 100))

Mean Accuracy: 85.794%
Standard Deviation: 0.053%
Min Accuracy: 75.000%
Max Accuracy: 96.000%


<table border = "1">
  <thead>
    <th colspan = "3">XGB Cross-Validation Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Mean</b></td>
      <td>85.794%</td>
      <td>92.496%</td>
    </tr>
    <tr>
      <td><b>Std.</b></td>
      <td>0.053%</td>
      <td>0.047%</td>
    </tr>
    <tr>
      <td><b>Min</b></td>
      <td>75.000%</td>
      <td>82.867%</td>
    </tr>
    <tr>
      <td><b>Max</b></td>
      <td>96.000%</td>
      <td>100.000%</td>
    </tr>
  </tbody>
</table>

In [13]:
# Apply Grid Search for parameter tuning

# xgb_params = {'max_depth': [1, 2, 3, 4, 5], 
#               'learning_rate': [0.05, 0.1, 0.15],
#               'colsample_bytree': [0.3, 0.5, 0.7],
#               'n_estimators': [25, 50, 75, 100, 150, 200],
#               'min_child_weight': [1, 2, 4, 5, 8, 10],
#               'gamma': [0, 0.1, 0.2, 0.3],
#               'subsample': [0.6, 0.8, 1.0],
#               'random_state': [0, 1]}

# Accuracy scoring
xgb_params = {'max_depth': [1], 
              'learning_rate': [0.16, 0.17, 0.18],
              'colsample_bytree': [0.31, 0.32],
              'min_child_weight': [11],
              'n_estimators': [26, 27, 28],
              'gamma': [0],
              'subsample': [0.9, 1.0, 1.1],
              'random_state': [0, 1]}

# ROC_AUC scoring
# xgb_params = {'max_depth': [1], 
#               'learning_rate': [0.1],
#               'colsample_bytree': [0.16],
#               'min_child_weight': [9, 10, 11, 12],
#               'n_estimators': [25],
#               'gamma': [0],
#               'subsample': [0.9, 1.0, 1.1],
#               'random_state': [0, 1]}

xgb_grid = GridSearchCV(xgb_model, xgb_params, scoring='accuracy', cv=cv, n_jobs=-1)
# xgb_grid = GridSearchCV(xgb_model, xgb_params, scoring='roc_auc', cv=cv, n_jobs=-1)
results = xgb_grid.fit(X_train, y_train)

# Summarize results
best_score = xgb_grid.best_score_
best_parameters = xgb_grid.best_params_
print("Best Score: %.3f%%" % (best_score * 100))
print("Best Parameters: ", best_parameters)
print('')

Best Score: 85.794%
Best Parameters:  {'colsample_bytree': 0.31, 'gamma': 0, 'learning_rate': 0.17, 'max_depth': 1, 'min_child_weight': 11, 'n_estimators': 28, 'random_state': 0, 'subsample': 1.0}



/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py", line 732, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py", line 216, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/training.py",

<table border = "1">
  <thead>
    <th colspan = "3">XGB Grid Search Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Best Score</b></td>
      <td>85.794%</td>
      <td>92.496%</td>
    </tr>
    <tr>
      <td colspan="3"><b>Best Parameters:</b></td>
    </tr>
    <tr>
      <td><b>max_depth</b></td>
      <td>1</td>
      <td>1</td>
    </tr>
    <tr>
      <td><b>min_child_weight</b></td>
      <td>11</td>
      <td>10</td>
    </tr>
    <tr>
      <td><b>gamma</b></td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <td><b>colsample_bytree</b></td>
      <td>0.31</td>
      <td>0.16</td>
    </tr>
    <tr>
      <td><b>subsample</b></td>
      <td>1.0</td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><b>learning_rate</b></td>
      <td>0.17</td>
      <td>0.1</td>
    </tr>
    <tr>
      <td><b>n_estimators</b></td>
      <td>28</td>
      <td>25</td>
    </tr>
    <tr>
      <td><b>random_state</b></td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

## 5. Final Results

<table border = "1">
  <thead>
    <th colspan = "3">XGB Model Final Scores</th>
  </thead>
    <tr>
      <td><b></b></td>
      <td><b>Pred.</b></td>
      <td><b>Val.</b></td>
    </tr>
    <tr>
      <td><b>Accuracy (%):</b></td>
      <td>85.794</td>
      <td>86.885</td>
    </tr>
    <tr>
      <td><b>ROC AUC (%):</b></td>
      <td>92.496</td>
      <td>86.710</td>
    </tr>
  </tbody>
</table>